In [42]:
import io
import json
import os
import sys
import random
import csv
import subprocess
import numpy as np
import pandas as pd
import tifffile as tiff
from PIL import Image
from math import sqrt, exp, log
from matplotlib import cm
from matplotlib import pyplot as plt
from pathlib import Path
from tqdm import tqdm
import re
import shutil
from flir_image_extractor import FlirImageExtractor
from scipy.ndimage import zoom

In [43]:
os.chdir(sys.path[0])

# /Volumes/SANDISK64GB/GitHub/Tesi_magistrale_Bugin/data_source
data_base_folder = os.path.join(
    '..', 'Tesi_magistrale_bugin', 'data_source', 'archivio_foto_tesi')

In [5]:
fie = FlirImageExtractor(exiftool_path='exiftool')

for current_folder, _, _ in os.walk(data_base_folder):
    jpg_file_list = sorted([os.path.join(current_folder, name) for name in os.listdir(current_folder) if
                            (not name.startswith('.')) and
                            name.endswith('.jpg') and
                            os.path.isfile(os.path.join(current_folder, name))])
    for jpg_file in jpg_file_list:
        processing_ok = True
        try:
            fie.process_image(jpg_file)
        except Exception as e:
            print(f"An error occurred calling fie.process_image(): {e}")
            processing_ok = False
        try:
            meta = fie.get_metadata()
        except Exception as e:
            print(f"An error occurred calling fie.get_metadata(): {e}")
            processing_ok = False
        try:
            thermal_channel = fie.get_thermal_np()
        except Exception as e:
            print(f"An error occurred calling fie.get_thermal_np(): {e}")
            processing_ok = False
        try:
            rgb_channels = fie.get_rgb_np()
        except Exception as e:
            print(f"An error occurred calling fie.get_rgb_np(): {e}")
            processing_ok = False
    
        if processing_ok:
            dest_npy_file = os.path.join(
                os.path.join('archivio_foto_tesi',
                             'pure_rgb_matrices_(480,640,3)'),
                f'{Path(jpg_file).stem}.npy')
            np.save(dest_npy_file, rgb_channels)
            dest_npy_file = os.path.join(
                os.path.join('archivio_foto_tesi',
                            'pure_thermal_matrices_(60,80)'),
                f'{Path(jpg_file).stem}.npy')
            np.save(dest_npy_file, thermal_channel)
            dest_json = os.path.join(
                os.path.join('archivio_foto_tesi',
                             'metadata'),
                f'{Path(jpg_file).stem}.json')
            with open(dest_json, 'w') as json_file:
                json.dump(meta, json_file, indent=4)


In [40]:
def resize(thermal_channel, dest_shape, metadata):
    Real2IR = metadata['Real2IR']
    OffsetX = metadata['OffsetX']
    OffsetY = metadata['OffsetY']

    # Define grid for interpolation
    # order=1 corresponds to linear interpolation
    H_orig, W_orig = thermal_channel.shape
    H_dest, W_dest, _ = dest_shape
    thermal_channel_scaled = zoom(thermal_channel, zoom=(H_dest/(Real2IR*H_orig), W_dest/(Real2IR*W_orig)), order=1)
    H_scaled, W_scaled = thermal_channel_scaled.shape
    thermal_channel_new_with_nan = np.full(
        (H_dest, W_dest), np.nan, dtype=np.float64)

    # TODO: modify the content of thermal_channel_new_with_nan to place 
    #       thermal_channel_scaled inside thermal_channel_new_with_nan so that the center of
    #       thermal_channel_scaled corresponds to the 
    # Place the rescaled thermal image into the new array at the correct location
    start_x = int((W_dest-W_scaled)/2) + OffsetX
    end_x = start_x + W_scaled
    start_y = int((H_dest-H_scaled)/2) + OffsetY
    end_y = start_y + H_scaled
    thermal_channel_new_with_nan[start_y:end_y,
                                 start_x:end_x] = thermal_channel_scaled


    return thermal_channel_new_with_nan

In [41]:
fie = FlirImageExtractor(exiftool_path='exiftool')

stop = False
counter = 0

for current_folder, _, _ in os.walk(data_base_folder):
    jpg_file_list = sorted([os.path.join(current_folder, name) for name in os.listdir(current_folder) if
                            (not name.startswith('.')) and
                            name.endswith('.jpg') and
                            os.path.isfile(os.path.join(current_folder, name))])

    for jpg_file in jpg_file_list:
        
        counter += 1

        metadata = None
        rgb_channels_norm = None
        thermal_channel_norm = None

        processing_ok = True
        try:
            fie.process_image(jpg_file)
        except Exception as e:
            print(f"An error occurred calling fie.process_image(): {e}")
            processing_ok = False
        try:
            metadata = fie.get_metadata()
        except Exception as e:
            print(f"An error occurred calling fie.get_metadata(): {e}")
            processing_ok = False
        try:
            rgb_channels = fie.get_rgb_np()
            rgb_channels_norm = (rgb_channels.astype(np.float32))/255.0
        except Exception as e:
            print(f"An error occurred calling fie.get_rgb_np(): {e}")
            processing_ok = False
        try:
            thermal_channel = fie.get_thermal_np()
        except Exception as e:
            print(f"An error occurred calling fie.get_thermal_np(): {e}")
            processing_ok = False

        if processing_ok:
            min_temp = np.min(thermal_channel)
            max_temp = np.max(thermal_channel)
            delta_temp = abs(max_temp-min_temp)
            thermal_channel_norm = (thermal_channel.astype(
                np.float32) - min_temp)/delta_temp
            # thermal_channel has size (60, 80)
            # rgb_channels_norm has shape (480, 640, 3)
            # thermal_channel_norm is resized to (H, W), with 60 < H < 480 and 80 < W < 640
            # finally thermal_channel_norm is put inside an outer frame of np.nan values
            # to resize it to (480, 640)
            thermal_channel_norm_resized = resize(
                thermal_channel_norm, rgb_channels_norm.shape, metadata)
            # Replace np.nan values with min_temp
            #thermal_channel_norm_resized = np.where(
            #    np.isnan(thermal_channel_norm_resized), min_temp, thermal_channel_norm_resized)
            cmap = plt.colormaps.get_cmap('rainbow')  # or 'nipy_spectral'
            thermal_channel_norm_as_rgb = cmap(
                thermal_channel_norm_resized)[:, :, :3] #this is normalized
            alpha = 0.25
            thermally_enhanced_rgb = (1-alpha)*rgb_channels_norm + \
                alpha * thermal_channel_norm_as_rgb
            ###############################################################
            """
            plt.title(f'{Path(jpg_file).stem} - rgb_channels_norm')
            plt.imshow(rgb_channels_norm)
            plt.show()
            plt.title(f'{Path(jpg_file).stem} - thermal_channel_norm_as_rgb')
            plt.imshow(thermal_channel_norm_as_rgb)
            plt.show()
            
            plt.title(f'{Path(jpg_file).stem} - thermally_enhanced_rgb')
            plt.imshow(thermally_enhanced_rgb)
            plt.show()
            """
            ###############################################################
            
            image = Image.fromarray((thermally_enhanced_rgb*255).astype(
                np.uint8))  # Ensure the array is of type uint8
            dest_jpg_file = os.path.join(
                os.path.join('archivio_foto_tesi',
                            'temperature_enhanced_images'),
                f'{Path(jpg_file).stem}.jpg')
            image.save(dest_jpg_file)
                
            
            

In [47]:
mask_data_folder = os.path.join(
    '..', 'Tesi_magistrale_bugin', 'elaborated_data', 'prediction_all_images_from_apeer')

ometiff_file_list = sorted([os.path.join(mask_data_folder, name) for name in os.listdir(mask_data_folder) if
                        (not name.startswith('.')) and
                        name.endswith('.ome.tiff') and
                        os.path.isfile(os.path.join(mask_data_folder, name))])

for ometiff_file in ometiff_file_list:    
    try:
        apeer_prediction_mask = tiff.imread(ometiff_file)
    except Exception as e:
        print(f'{e}. An error occurred calling tiff.imread({ometiff_file})')
        continue

    new_stem = (Path(ometiff_file).stem).replace('_finalprediction.ome', '')
    dest_npy_file = os.path.join(
        'archivio_foto_tesi',
        'APEER_PRED_binary_masks_cold_dirt(480,640)',
        f'{new_stem}.npy')
    # Save the binary mask to a npy file
    # la classe 'cold dirt' è la seconda immagine delle tre nel file apeer_prediction_file
    if not os.path.exists(dest_npy_file):
        np.save(dest_npy_file, apeer_prediction_mask[1, :, :])
    
    dest_npy_file = os.path.join(
        'archivio_foto_tesi',
        'APEER_PRED_binary_masks_hot_dirt(480,640)',
        f'{new_stem}.npy')
    # la classe 'hot dirt' è la terza immagine delle tre nel file apeer_prediction_file
    if not os.path.exists(dest_npy_file):
        np.save(dest_npy_file, apeer_prediction_mask[2, :, :])

not a TIFF file b''. An error occurred calling tiff.imread(../Tesi_magistrale_bugin/elaborated_data/prediction_all_images_from_apeer/FLIR1979_finalprediction.ome.tiff)
not a TIFF file b''. An error occurred calling tiff.imread(../Tesi_magistrale_bugin/elaborated_data/prediction_all_images_from_apeer/FLIR2831_finalprediction.ome.tiff)
not a TIFF file b''. An error occurred calling tiff.imread(../Tesi_magistrale_bugin/elaborated_data/prediction_all_images_from_apeer/FLIR3523_finalprediction.ome.tiff)
not a TIFF file b''. An error occurred calling tiff.imread(../Tesi_magistrale_bugin/elaborated_data/prediction_all_images_from_apeer/FLIR3826_finalprediction.ome.tiff)
not a TIFF file b''. An error occurred calling tiff.imread(../Tesi_magistrale_bugin/elaborated_data/prediction_all_images_from_apeer/FLIR4157_finalprediction.ome.tiff)
not a TIFF file b''. An error occurred calling tiff.imread(../Tesi_magistrale_bugin/elaborated_data/prediction_all_images_from_apeer/FLIR5048_finalprediction.om

In [ ]:
mask_data_folder = os.path.join(
    'archivio_foto_tesi', 'APEER_PRED_binary_masks_cold_dirt(480,640)')

npy_file_list = sorted([os.path.join(mask_data_folder, name) for name in os.listdir(mask_data_folder) if
                            (not name.startswith('.')) and
                            name.endswith('.npy') and
                            os.path.isfile(os.path.join(mask_data_folder, name))])

i = 4
npy_file = npy_file_list[i]
